In [1]:
import torch
from torchvision.models import vgg19
import torch.nn as nn

In [10]:
class V2A(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = vgg19(pretrained=True)
        self.rnn = nn.LSTMCell(input_size=2000, hidden_size=256)
        self.sound_features = nn.Linear(in_features=256, out_features=42)
    
    def forward(self, rgb, st):
        hn = None
        cn = None
        sounds = []
        for t in range(rgb.shape[1]):
            cnn_feat_rgb = self.encoder(rgb[:, t, :, :, :])
            cnn_feat_st = self.encoder(st[:, t, :, :, :])
            comb_feat = torch.concat([cnn_feat_rgb, cnn_feat_st], dim=1)
            if hn is None:
                hn, cn = self.rnn(comb_feat)
            else:
                hn, cn = self.rnn(comb_feat, (hn, cn))
            sound_features = self.sound_features(hn)
            print(sound_features.shape)
            sounds.append(sound_features)
        return torch.tensor(sounds)

In [11]:
model = V2A()

/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home2/aa_aanegola/miniconda3/envs/cv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
rgb = torch.zeros((4, 45, 3, 224, 224))
st = torch.zeros((4, 45, 3, 224, 224))
model(rgb, st)

torch.Size([4, 42])
torch.Size([4, 42])
torch.Size([4, 42])
torch.Size([4, 42])
torch.Size([4, 42])
torch.Size([4, 42])
torch.Size([4, 42])
torch.Size([4, 42])


: 

: 